In [ ]:
import numpy as np, random
from pathlib import Path
random.seed(42); np.random.seed(42)

def load_problem(path: str|Path) -> np.ndarray:
    M = np.load(path)
    assert M.ndim == 2 and M.shape[0] == M.shape[1], "Matrix must be square"
    return M

def tour_cost(tour: np.ndarray, M: np.ndarray) -> float:
    c = float(M[tour[-1], tour[0]])
    for i in range(len(tour)-1):
        c += float(M[tour[i], tour[i+1]])
    return c

def initialize_population(M: np.ndarray, size: int) -> list[np.ndarray]:
    n = M.shape[0]
    pop = []
    # metà nearest-neighbor multi-start “povero”
    nn_count = size // 2
    for _ in range(nn_count):
        start = random.randrange(n)
        unv = set(range(n)); unv.remove(start)
        tour = [start]
        cur = start
        while unv:
            nxt = min(unv, key=lambda j: M[cur, j])
            tour.append(nxt); unv.remove(nxt); cur = nxt
        pop.append(np.array(tour, dtype=int))
    # metà random
    for _ in range(size - nn_count):
        t = np.arange(n, dtype=int)
        np.random.shuffle(t)
        pop.append(t)
    return pop

def tournament_select(pop: list[np.ndarray], costs: np.ndarray, k: int=3) -> np.ndarray:
    idxs = np.random.choice(len(pop), size=k, replace=False)
    best_i = idxs[np.argmin(costs[idxs])]
    return pop[best_i]


In [ ]:
def cx_ox(p1: np.ndarray, p2: np.ndarray) -> np.ndarray:
    n = len(p1)
    i, j = sorted(np.random.choice(n, size=2, replace=False))
    child = np.full(n, -1, dtype=int)
    # copia segmento da p1
    child[i:j+1] = p1[i:j+1]
    used = set(child[i:j+1])
    # riempi da p2 nell'ordine, saltando i già usati
    pos = (j + 1) % n
    for gene in list(p2) * 2:  # wrap-around
        if gene not in used:
            child[pos] = gene
            used.add(gene)
            pos = (pos + 1) % n
            if pos == i: break
    return child

def mut_swap(t: np.ndarray) -> np.ndarray:
    n = len(t); a, b = np.random.choice(n, size=2, replace=False)
    out = t.copy(); out[a], out[b] = out[b], out[a]; return out

def mut_insert(t: np.ndarray) -> np.ndarray:
    n = len(t); a, b = np.random.choice(n, size=2, replace=False)
    out = np.delete(t, a)
    out = np.insert(out, b, t[a])
    return out

def mut_inversion(t: np.ndarray) -> np.ndarray:
    n = len(t); i, j = sorted(np.random.choice(n, size=2, replace=False))
    out = t.copy(); out[i:j+1] = out[i:j+1][::-1]; return out

def mutate(t: np.ndarray) -> np.ndarray:
    op = random.choice([mut_swap, mut_insert, mut_inversion])
    return op(t)


In [ ]:
def auto_params_for_n(n: int) -> dict:
    if n <= 80:
        return dict(pop_size=120, generations=300, cx_rate=0.9, mut_rate=0.20, elitism=2)
    elif n <= 300:
        return dict(pop_size=100, generations=220, cx_rate=0.9, mut_rate=0.18, elitism=2)
    else:
        return dict(pop_size=80,  generations=180, cx_rate=0.9, mut_rate=0.15, elitism=2)

def run_ga_on_matrix(M: np.ndarray, pop_size:int, generations:int, cx_rate:float, mut_rate:float, elitism:int=2, seed:int=42):
    random.seed(seed); np.random.seed(seed)
    pop = initialize_population(M, pop_size)
    costs = np.array([tour_cost(t, M) for t in pop])
    best_i = int(np.argmin(costs)); best, best_cost = pop[best_i].copy(), float(costs[best_i])

    for _ in range(generations):
        # elitismo
        elite_idx = np.argsort(costs)[:elitism]
        new_pop = [pop[i].copy() for i in elite_idx]

        # riproduzione
        while len(new_pop) < pop_size:
            # selezione
            p1 = tournament_select(pop, costs, k=3)
            child = p1.copy()
            # crossover
            if np.random.rand() < cx_rate:
                p2 = tournament_select(pop, costs, k=3)
                child = cx_ox(p1, p2)
            # mutazione
            if np.random.rand() < mut_rate:
                child = mutate(child)
            new_pop.append(child)

        pop = new_pop
        costs = np.array([tour_cost(t, M) for t in pop])
        bi = int(np.argmin(costs))
        if costs[bi] < best_cost:
            best_cost = float(costs[bi]); best = pop[bi].copy()

    return best, best_cost

# Helper "one-shot" su un file
def solve_one_file(path: str|Path, seed: int=42):
    M = load_problem(path)
    n = M.shape[0]
    cfg = auto_params_for_n(n)
    best, cost = run_ga_on_matrix(M, seed=seed, **cfg)
    return dict(file=str(path), n=n, best_cost=cost, best_tour=best, cfg=cfg)
